In [1]:
import pandas as pd

pd.set_option('display.max_columns', 500)

In [2]:
data = pd.read_csv("data/processed_data_2022-06-13.csv")
data.head()

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,76,76,NaN,NaN,1987-01-27,88,191,NaN,NaN,NaN,0,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,11,12,14,15,13,30,20,50,39,43,71,48,53,64,30,68,12,36,22,13,47,21,42,16,14,17,77,73,70,75,78,46.0,30,30,30,27,28,28,28,27,31,31,31,30,31,31,31,30,29,31,31,31,29,29,30,30,30,29,76,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,76,76,NaN,NaN,1987-10-23,84,196,NaN,NaN,NaN,1,2,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,12,24,28,16,16,48,23,47,40,40,73,46,56,68,40,79,12,44,24,14,57,23,60,21,23,21,78,74,74,74,73,44.0,35,35,35,31,33,33,33,31,37,37,37,36,39,39,39,36,33,39,39,39,33,33,37,37,37,33,76,34,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,74,74,NaN,NaN,1988-03-17,84,187,NaN,NaN,NaN,1,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,6,10,7,14,11,22,12,20,21,27,63,43,51,59,27,72,5,20,6,5,31,16,56,9,13,13,76,71,68,74,78,20.0,24,24,24,17,19,19,19,17,22,22,22,21,23,23,23,21,21,24,24,24,21,20,24,24,24,20,74,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,72,72,NaN,NaN,1993-03-21,89,197,NaN,NaN,NaN,1,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,13,18,16,14,12,25,16,48,45,50,70,36,51,37,33,64,13,39,19,14,45,14,61,19,14,15,70,70,68,71,72,47.0,30,30,30,26,28,28,28,26,29,29,29,29,29,29,29,29,28,29,29,29,28,28,29,29,29,28,72,29,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,72,82,NaN,NaN,1999-02-25,82,192,NaN,NaN,NaN,1,2,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,7,14,10,11,12,26,18,19,22,33,38,26,55,69,26,64,8,27,13,7,55,11,55,25,13,14,74,74,73,75,76,20.0,23,23,23,19,21,21,21,19,25,25,25,23,27,27,27,23,21,26,26,26,21,21,27,27,27,21,73,23,0,0,0,0,0,0,0

In [3]:
def count_na(df: pd.DataFrame) -> None:
    nan_sum = df.isna().sum()
    display(nan_sum[nan_sum > 0])

count_na(data)

value_eur                74
wage_eur                 61
club_team_id             61
club_position            61
club_jersey_number       61
release_clause_eur     1176
pace                   2132
shooting               2132
passing                2132
dribbling              2132
defending              2132
physic                 2132
goalkeeping_speed     17107
dtype: int64

Analisando essa tabela com os valores nulos podemos perceber que alguns jogadores do dataset não possuem um time atual, isso explica
`wage_eur`, `club_team_id`, `club_position`, `club_jersey_number` terem a mesma quantidade de valores nulos (61),
porém `value_eur` deveria seguir esse mesmo padrão, assim podemos dizer que 13 (74 - 61) jogadores que estão ativos não possuem um valor,
portanto precisamos prever esses dados.

Também podemos perceber que temos 2132 jogadores não possuem `pace`, `shooting`, `passing`, `dribbling`, `defending`, `physic`. Vamos analisar esses jogadores

In [4]:
missing_main_attr_data = data[data.pace.isna()]

n_rows = missing_main_attr_data.shape[0]
n_gk = missing_main_attr_data.GK.sum()
print(f"porcentagem de goleiros {n_gk/n_rows * 100}%")
missing_main_attr_data

porcentagem de goleiros 100.0%


,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,76,76,NaN,NaN,1987-01-27,88,191,NaN,NaN,NaN,0,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,11,12,14,15,13,30,20,50,39,43,71,48,53,64,30,68,12,36,22,13,47,21,42,16,14,17,77,73,70,75,78,46.0,30,30,30,27,28,28,28,27,31,31,31,30,31,31,31,30,29,31,31,31,29,29,30,30,30,29,76,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,76,76,NaN,NaN,1987-10-23,84,196,NaN,NaN,NaN,1,2,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,12,24,28,16,16,48,23,47,40,40,73,46,56,68,40,79,12,44,24,14,57,23,60,21,23,21,78,74,74,74,73,44.0,35,35,35,31,33,33,33,31,37,37,37,36,39,39,39,36,33,39,39,39,33,33,37,37,37,33,76,34,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,74,74,NaN,NaN,1988-03-17,84,187,NaN,NaN,NaN,1,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,6,10,7,14,11,22,12,20,21,27,63,43,51,59,27,72,5,20,6,5,31,16,56,9,13,13,76,71,68,74,78,20.0,24,24,24,17,19,19,19,17,22,22,22,21,23,23,23,21,21,24,24,24,21,20,24,24,24,20,74,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,72,72,NaN,NaN,1993-03-21,89,197,NaN,NaN,NaN,1,3,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,13,18,16,14,12,25,16,48,45,50,70,36,51,37,33,64,13,39,19,14,45,14,61,19,14,15,70,70,68,71,72,47.0,30,30,30,26,28,28,28,26,29,29,29,29,29,29,29,29,28,29,29,29,28,28,29,29,29,28,72,29,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,72,82,NaN,NaN,1999-02-25,82,192,NaN,NaN,NaN,1,2,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,7,14,10,11,12,26,18,19,22,33,38,26,55,69,26,64,8,27,13,7,55,11,55,25,13,14,74,74,73,75,76,20.0,23,23,23,19,21,21,21,19,25,25,25,23,27,27,27,23,21,26,26,26,21,21,27,27,27,21,73,23,0,0,0,0,0,0,0

Como vimos que os goleiros não tem os atributos listados acima, mas possuem um overall em cada posição, seria interessante prever esses atributos.

In [5]:
missing_gk_speed_data = data[data.goalkeeping_speed.isna()]
sum_of_missing_data = missing_gk_speed_data.shape[0] + missing_main_attr_data.shape[0]
print(f"Jogadores sem gk speed + jogadores sem atributos primários: {sum_of_missing_data}")
print(f"Numero de jogadores: {data.shape[0]}")

Jogadores sem gk speed + jogadores sem atributos primários: 19239
Numero de jogadores: 19239


Assim podemos ver que todos os jogadores que não são goleiros não possuem `goalkeeping_speed`

## Prevendo valor dos jogadores
Nessa etapa vamos criar um modelo para prever o valor dos 13 jogadores que não possuem valor

In [6]:
players_with_club_and_no_value = data.query("not club_team_id.isna() and value_eur.isna()").copy()
players_with_club_and_no_value

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
11929,115909,https://sofifa.com/player/115909/ruben-castro-...,Rubén Castro,Rubén Castro Martín,69,69,NaN,3000.0,1981-06-27,68,169,100851.0,ST,7.0,1,3,3,2,0,0,NaN,70.0,70.0,61.0,72.0,29.0,61.0,60,71,62,70,68,67,45,69,72,69,86,75,81,69,85,62,61,68,54,37,77,74,75,68,36,12,14,14,8,15,8,12,NaN,69,69,69,69,69,69,69,69,69,69,69,69,63,63,63,69,52,49,49,49,52,49,45,45,45,49,20,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
11987,124344,https://sofifa.com/player/124344/ibrahim-oztur...,İ. Öztürk,İbrahim Öztürk,67,67,NaN,3000.0,1981-06-21,80,186,101006.0,CB,38.0,1,2,2,1,0,0,NaN,32.0,43.0,48.0,48.0,68.0,74.0,44,30,72,46,44,48,42,54,30,34,32,68,38,66,59,65,78,46,78,66,34,44,51,51,70,68,62,15,13,10,10,6,NaN,52,52,52,44,47,47,47,44,48,48,48,48,54,54,54,48,59,64,64,64,59,61,67,67,67,61,19,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
12264,153066,https://sofifa.com/player/153066/lucas-licht/2...,L. Licht,Lucas Matías Licht,69,69,NaN,5000.0,1981-04-06,72,174,101084.0,SUB,25.0,0,4,3,1,0,0,NaN,64.0,61.0,65.0,72.0,68.0,66.0,80,52,53,76,73,68,71,69,59,68,69,66,73,71,72,64,66,68,69,71,56,64,84,72,69,71,69,8,14,6,15,8,NaN,63,63,63,65,63,63,63,65,66,66,66,68,67,67,67,68,69,69,69,69,69,69,69,69,69,69,19,41,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0
13149,152912,https://sofifa.com/player/152912/jose-sand/220002,J. Sand,José Gustavo Sand,75,75,NaN,13000.0,1980-07-17,79,182,110395.0,LS,9.0,1,4,3,1,0,0,NaN,51.0,79.0,62.0,68.0,37.0,70.0,53,81,77,67,63,63,46,75,49,53,38,81,67,77,71,41,83,73,73,33,83,69,85,85,45,21,25,11,16,15,9,14,NaN,75,75,75,68,73,73,73,68,72,72,72,68,65,65,65,68,51,54,54,54,51,49,53,53,53,49,22,41,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13177,110381,https://sofifa.com/player/110381/maximiliano-r...,M. Rodríguez,Maximiliano Rubén Rodríguez,73,73,NaN,10000.0,1981-01-02,74,173,110396.0,SUB,11.0,1,4,4,3,0,0,NaN,64.0,74.0,76.0,75.0,44.0,64.0,72,73,61,75,76,71,75,76,64,64,72,72,70,77,67,46,70,75,73,47,76,76,77,86,57,30,26,

In [7]:
data_without_players_with_club_and_no_value = data.drop(players_with_club_and_no_value.index)
data_to_model = data_without_players_with_club_and_no_value.query("not value_eur.isna()").copy()
data_to_model.drop(columns=["sofifa_id", "player_url", "short_name", "long_name", "dob", "club_position"], inplace=True)
data_to_model

,overall,potential,value_eur,wage_eur,weight_kg,height_cm,club_team_id,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
8,83,84,28000000.0,87000.0,83,190,1.0,1.0,1,3,1,3,0,0,55300000.0,NaN,NaN,NaN,NaN,NaN,NaN,9,9,13,16,9,8,48,35,55,56,52,78,44,57,73,43,70,14,28,25,7,56,23,62,27,15,18,85,80,76,83,85,55.0,35,35,35,32,35,35,35,32,41,41,41,39,43,43,43,39,37,41,41,41,37,36,36,36,36,36,84,30,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,74,79,5000000.0,36000.0,77,188,1.0,32.0,1,3,1,1,0,0,9900000.0,NaN,NaN,NaN,NaN,NaN,NaN,15,14,15,18,17,16,33,28,47,50,53,70,49,52,66,35,59,15,35,20,11,47,25,55,18,16,15,75,72,69,71,76,48.0,32,32,32,30,31,31,31,30,34,34,34,33,34,34,34,33,30,33,33,33,30,30,30,30,30,30,75,24,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
10,61,77,750000.0,3000.0,83,196,1.0,33.0,1,2,1,1,0,0,1900000.0,NaN,NaN,NaN,NaN,NaN,NaN,10,6,12,9,12,20,17,17,44,44,40,58,31,41,50,29,50,7,20,11,6,30,19,30,13,13,12,60,58,54,62,63,44.0,24,24,24,21,22,22,22,21,25,25,25,25,24,24,24,25,24,24,24,24,24,24,24,24,24,24,62,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
11,84,85,33500000.0,81000.0,88,195,2.0,1.0,1,4,1,3,0,0,66200000.0,NaN,NaN,NaN,NaN,NaN,NaN,22,16,26,13,22,20,51,35,59,57,49,80,49,62,60,40,70,16,37,13,12,66,42,60,11,20,20,85,83,82,83,85,58.0,39,39,39,35,37,37,37,35,43,43,43,41,45,45,45,41,36,40,40,40,36,36,35,35,35,36,85,29,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
12,72,73,1900000.0,31000.0,80,188,2.0,12.0,0,2,1,1,0,0,3800000.0,NaN,NaN,NaN,NaN,NaN,NaN,11,11,12,11,12,11,25,22,53,47,37,67,59,53,56,39,61,11,22,22,16,42,41,30,12,12,11,73,70,70,70,72,51.0,31,31,31,27,29,29,29,27,32,32,32,31,32,32,32,31,29,31,31,31,29,29,27,27,27,29,73,29,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [8]:
from sklearn.impute import SimpleImputer
import numpy as np

target = data_to_model.value_eur
features = data_to_model.drop(columns=["value_eur"])
# features = features.fillna(0)
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
mean_imputer.fit(features)
imputed_features = mean_imputer.transform(features)

imputed_features

array([[8.3e+01, 8.4e+01, 8.7e+04, ..., 0.0e+00, 0.0e+00, 0.0e+00],
       [7.4e+01, 7.9e+01, 3.6e+04, ..., 0.0e+00, 0.0e+00, 0.0e+00],
       [6.1e+01, 7.7e+01, 3.0e+03, ..., 0.0e+00, 0.0e+00, 0.0e+00],
       ...,
       [5.5e+01, 6.5e+01, 5.0e+02, ..., 0.0e+00, 0.0e+00, 1.0e+00],
       [5.5e+01, 6.7e+01, 5.0e+02, ..., 0.0e+00, 0.0e+00, 1.0e+00],
       [5.5e+01, 6.6e+01, 5.0e+02, ..., 0.0e+00, 0.0e+00, 0.0e+00]])

In [9]:
# Normalizando os dados
# from sklearn.preprocessing import StandardScaler
#
# standerd_scaler =  StandardScaler()
# standardized_features = standerd_scaler.fit_transform(imputed_features)
# standardized_features

In [10]:
# train test split
from sklearn.model_selection import train_test_split

features_train, features_test, target_train, target_test = train_test_split(imputed_features, target, test_size=0.2)

## Comparando modelos diferentes

In [11]:
from sklearn.dummy import DummyRegressor

dummy = DummyRegressor()
dummy_model = dummy.fit(features_train, target_train)
dummy_score = dummy_model.score(features_test, target_test)

In [12]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
linear_regression_model = linear_regression.fit(features_train, target_train)
linear_regression_score = linear_regression_model.score(features_test, target_test)

In [13]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor(random_state=0, n_jobs=-1, oob_score=True)
random_forest_model = random_forest.fit(features_train, target_train)
random_forest_score = random_forest_model.score(features_test, target_test)

In [14]:
score_valuation_text = f"""Score de cada modelo:
    Média {dummy_score:.4f}
    Regressão Linear {linear_regression_score:.4f}
    Floresta Aleatória {random_forest_score:.4f}
"""
print(score_valuation_text)

Score de cada modelo:
    Média -0.0007
    Regressão Linear 0.9862
    Floresta Aleatória 0.9730



### Preenchendo os dados a partir do modelo criado

In [15]:
data_to_fill = players_with_club_and_no_value.drop(columns=["sofifa_id", "player_url", "short_name", "long_name", "dob", "club_position", "value_eur"])

data_to_fill = mean_imputer.transform(data_to_fill)
predicted_values = random_forest_model.predict(data_to_fill)
predicted_values

array([ 858750.,  828500.,  890250., 3823000., 2322250.,  231150.,
        785750., 3543000.,  841000.,  216550.,  239000.,  498000.,
        379250.])

In [16]:
players_with_club_and_no_value.value_eur = predicted_values
players_with_club_and_no_value

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
11929,115909,https://sofifa.com/player/115909/ruben-castro-...,Rubén Castro,Rubén Castro Martín,69,69,858750.0,3000.0,1981-06-27,68,169,100851.0,ST,7.0,1,3,3,2,0,0,NaN,70.0,70.0,61.0,72.0,29.0,61.0,60,71,62,70,68,67,45,69,72,69,86,75,81,69,85,62,61,68,54,37,77,74,75,68,36,12,14,14,8,15,8,12,NaN,69,69,69,69,69,69,69,69,69,69,69,69,63,63,63,69,52,49,49,49,52,49,45,45,45,49,20,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
11987,124344,https://sofifa.com/player/124344/ibrahim-oztur...,İ. Öztürk,İbrahim Öztürk,67,67,828500.0,3000.0,1981-06-21,80,186,101006.0,CB,38.0,1,2,2,1,0,0,NaN,32.0,43.0,48.0,48.0,68.0,74.0,44,30,72,46,44,48,42,54,30,34,32,68,38,66,59,65,78,46,78,66,34,44,51,51,70,68,62,15,13,10,10,6,NaN,52,52,52,44,47,47,47,44,48,48,48,48,54,54,54,48,59,64,64,64,59,61,67,67,67,61,19,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
12264,153066,https://sofifa.com/player/153066/lucas-licht/2...,L. Licht,Lucas Matías Licht,69,69,890250.0,5000.0,1981-04-06,72,174,101084.0,SUB,25.0,0,4,3,1,0,0,NaN,64.0,61.0,65.0,72.0,68.0,66.0,80,52,53,76,73,68,71,69,59,68,69,66,73,71,72,64,66,68,69,71,56,64,84,72,69,71,69,8,14,6,15,8,NaN,63,63,63,65,63,63,63,65,66,66,66,68,67,67,67,68,69,69,69,69,69,69,69,69,69,69,19,41,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0
13149,152912,https://sofifa.com/player/152912/jose-sand/220002,J. Sand,José Gustavo Sand,75,75,3823000.0,13000.0,1980-07-17,79,182,110395.0,LS,9.0,1,4,3,1,0,0,NaN,51.0,79.0,62.0,68.0,37.0,70.0,53,81,77,67,63,63,46,75,49,53,38,81,67,77,71,41,83,73,73,33,83,69,85,85,45,21,25,11,16,15,9,14,NaN,75,75,75,68,73,73,73,68,72,72,72,68,65,65,65,68,51,54,54,54,51,49,53,53,53,49,22,41,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13177,110381,https://sofifa.com/player/110381/maximiliano-r...,M. Rodríguez,Maximiliano Rubén Rodríguez,73,73,2322250.0,10000.0,1981-01-02,74,173,110396.0,SUB,11.0,1,4,4,3,0,0,NaN,64.0,74.0,76.0,75.0,44.0,64.0,72,73,61,75,76,71,75,76,64,64,72,72,70,77,67,46,70,75,

In [17]:
data.loc[players_with_club_and_no_value.index, "value_eur"] = players_with_club_and_no_value.value_eur
data.loc[players_with_club_and_no_value.index]

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
11929,115909,https://sofifa.com/player/115909/ruben-castro-...,Rubén Castro,Rubén Castro Martín,69,69,858750.0,3000.0,1981-06-27,68,169,100851.0,ST,7.0,1,3,3,2,0,0,NaN,70.0,70.0,61.0,72.0,29.0,61.0,60,71,62,70,68,67,45,69,72,69,86,75,81,69,85,62,61,68,54,37,77,74,75,68,36,12,14,14,8,15,8,12,NaN,69,69,69,69,69,69,69,69,69,69,69,69,63,63,63,69,52,49,49,49,52,49,45,45,45,49,20,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
11987,124344,https://sofifa.com/player/124344/ibrahim-oztur...,İ. Öztürk,İbrahim Öztürk,67,67,828500.0,3000.0,1981-06-21,80,186,101006.0,CB,38.0,1,2,2,1,0,0,NaN,32.0,43.0,48.0,48.0,68.0,74.0,44,30,72,46,44,48,42,54,30,34,32,68,38,66,59,65,78,46,78,66,34,44,51,51,70,68,62,15,13,10,10,6,NaN,52,52,52,44,47,47,47,44,48,48,48,48,54,54,54,48,59,64,64,64,59,61,67,67,67,61,19,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
12264,153066,https://sofifa.com/player/153066/lucas-licht/2...,L. Licht,Lucas Matías Licht,69,69,890250.0,5000.0,1981-04-06,72,174,101084.0,SUB,25.0,0,4,3,1,0,0,NaN,64.0,61.0,65.0,72.0,68.0,66.0,80,52,53,76,73,68,71,69,59,68,69,66,73,71,72,64,66,68,69,71,56,64,84,72,69,71,69,8,14,6,15,8,NaN,63,63,63,65,63,63,63,65,66,66,66,68,67,67,67,68,69,69,69,69,69,69,69,69,69,69,19,41,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0
13149,152912,https://sofifa.com/player/152912/jose-sand/220002,J. Sand,José Gustavo Sand,75,75,3823000.0,13000.0,1980-07-17,79,182,110395.0,LS,9.0,1,4,3,1,0,0,NaN,51.0,79.0,62.0,68.0,37.0,70.0,53,81,77,67,63,63,46,75,49,53,38,81,67,77,71,41,83,73,73,33,83,69,85,85,45,21,25,11,16,15,9,14,NaN,75,75,75,68,73,73,73,68,72,72,72,68,65,65,65,68,51,54,54,54,51,49,53,53,53,49,22,41,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13177,110381,https://sofifa.com/player/110381/maximiliano-r...,M. Rodríguez,Maximiliano Rubén Rodríguez,73,73,2322250.0,10000.0,1981-01-02,74,173,110396.0,SUB,11.0,1,4,4,3,0,0,NaN,64.0,74.0,76.0,75.0,44.0,64.0,72,73,61,75,76,71,75,76,64,64,72,72,70,77,67,46,70,75,

In [18]:
count_na(data)

value_eur                61
wage_eur                 61
club_team_id             61
club_position            61
club_jersey_number       61
release_clause_eur     1176
pace                   2132
shooting               2132
passing                2132
dribbling              2132
defending              2132
physic                 2132
goalkeeping_speed     17107
dtype: int64

In [19]:
# Removendo jogadores não ativos
data = data.query("not club_team_id.isna()").copy()

count_na(data)

release_clause_eur     1115
pace                   2124
shooting               2124
passing                2124
dribbling              2124
defending              2124
physic                 2124
goalkeeping_speed     17054
dtype: int64

In [20]:
# TODO acidionar .joblib ou .pickle do modelo

# Prevendo pace, shooting, passing, dribbling, defending, physic dos goleiros

In [21]:
gk_data = data.query("GK == 1").copy()
gk_data

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
8,192563,https://sofifa.com/player/192563/bernd-leno/22...,B. Leno,Bernd Leno,83,84,28000000.0,87000.0,1992-03-04,83,190,1.0,GK,1.0,1,3,1,3,0,0,55300000.0,NaN,NaN,NaN,NaN,NaN,NaN,9,9,13,16,9,8,48,35,55,56,52,78,44,57,73,43,70,14,28,25,7,56,23,62,27,15,18,85,80,76,83,85,55.0,35,35,35,32,35,35,35,32,41,41,41,39,43,43,43,39,37,41,41,41,37,36,36,36,36,36,84,30,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,233934,https://sofifa.com/player/233934/aaron-ramsdal...,A. Ramsdale,Aaron Ramsdale,74,79,5000000.0,36000.0,1998-05-14,77,188,1.0,SUB,32.0,1,3,1,1,0,0,9900000.0,NaN,NaN,NaN,NaN,NaN,NaN,15,14,15,18,17,16,33,28,47,50,53,70,49,52,66,35,59,15,35,20,11,47,25,55,18,16,15,75,72,69,71,76,48.0,32,32,32,30,31,31,31,30,34,34,34,33,34,34,34,33,30,33,33,33,30,30,30,30,30,30,75,24,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
10,246671,https://sofifa.com/player/246671/arthur-okonkw...,A. Okonkwo,Arthur Okonkwo,61,77,750000.0,3000.0,2001-09-09,83,196,1.0,RES,33.0,1,2,1,1,0,0,1900000.0,NaN,NaN,NaN,NaN,NaN,NaN,10,6,12,9,12,20,17,17,44,44,40,58,31,41,50,29,50,7,20,11,6,30,19,30,13,13,12,60,58,54,62,63,44.0,24,24,24,21,22,22,22,21,25,25,25,25,24,24,24,25,24,24,24,24,24,24,24,24,24,24,62,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
11,202811,https://sofifa.com/player/202811/emiliano-mart...,E. Martínez,Damián Emiliano Martínez,84,85,33500000.0,81000.0,1992-09-02,88,195,2.0,GK,1.0,1,4,1,3,0,0,66200000.0,NaN,NaN,NaN,NaN,NaN,NaN,22,16,26,13,22,20,51,35,59,57,49,80,49,62,60,40,70,16,37,13,12,66,42,60,11,20,20,85,83,82,83,85,58.0,39,39,39,35,37,37,37,35,43,43,43,41,45,45,45,41,36,40,40,40,36,36,35,35,35,36,85,29,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
12,192317,https://sofifa.com/player/192317/jed-steer/220002,J. Steer,Jed Steer,72,73,1900000.0,31000.0,1992-09-23,80,188,2.0,SUB,12.0,0,2,1,1,0,0,3800000.0,NaN,NaN,NaN,NaN,NaN,NaN,11,11,12,11,12,11,25,22,53,47,37,67,59,53,56,39,61,11,22,22,16,42,41,30,12,12,11,73,70,70,70,72,51.0,31,31,31,

In [22]:
data_without_gk = data.query("GK == 0").copy()
data_without_gk

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
2185,188567,https://sofifa.com/player/188567/pierre-emeric...,P. Aubameyang,Pierre-Emerick Emiliano François Aubameyang,85,85,43500000.0,145000.0,1989-06-18,80,187,1.0,ST,14.0,1,4,4,3,0,0,82700000.0,89.0,85.0,75.0,80.0,35.0,69.0,76,89,73,79,81,76,64,81,87,91,77,86,67,82,75,79,74,78,43,48,90,77,83,87,20,29,36,6,9,15,9,9,NaN,85,85,85,82,83,83,83,82,84,84,84,84,77,77,77,84,67,61,61,61,67,63,54,54,54,63,20,32,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1
2186,209989,https://sofifa.com/player/209989/thomas-partey...,T. Partey,Thomas Partey,83,83,36500000.0,115000.0,1993-06-13,77,185,1.0,RDM,5.0,1,3,3,2,0,0,69400000.0,69.0,71.0,82.0,81.0,80.0,84.0,74,66,68,84,69,70,85,83,63,74,75,82,50,86,83,83,84,78,84,81,70,85,49,86,80,84,80,11,13,9,9,7,NaN,77,77,77,78,78,78,78,78,82,82,82,81,83,83,83,81,82,83,83,83,82,81,83,83,83,81,20,29,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2187,193301,https://sofifa.com/player/193301/alexandre-lac...,A. Lacazette,Alexandre Lacazette,82,82,30000000.0,110000.0,1991-05-28,73,175,1.0,SUB,9.0,1,4,4,3,1,0,57000000.0,75.0,82.0,74.0,82.0,44.0,72.0,68,83,75,83,74,74,70,82,77,73,81,82,82,87,76,66,75,78,73,42,82,75,79,80,39,45,32,11,6,9,5,6,NaN,82,82,82,79,81,81,81,79,82,82,82,81,77,77,77,81,64,65,65,65,64,62,60,60,60,62,18,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2188,222665,https://sofifa.com/player/222665/martin-odegaa...,M. Ødegaard,Martin Ødegaard,82,88,49000000.0,89000.0,1998-12-17,68,178,1.0,CAM,8.0,0,2,5,3,0,0,102900000.0,77.0,74.0,83.0,84.0,58.0,62.0,84,73,52,85,81,83,81,85,80,75,85,74,83,76,46,78,56,77,63,65,76,86,63,76,65,49,51,14,15,15,8,10,NaN,77,77,77,81,79,79,79,81,84,84,84,84,82,82,82,84,75,73,73,73,75,71,63,63,63,71,22,23,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2189,226110,https://sofifa.com/player/226110/nicolas-pepe/...,N. Pépé,Nicolas Pépé,81,82,30500000.0,95000.0,1995-05-29,73,183,1.0,RM,19.0,0,2,4,2,0,0,60200000.0,88.0,79.0,75.0,83.0,36.0,65.0,77,79,57,85,81,79,66,81,88,8

In [23]:
columns_to_predict = ["pace", "shooting", "passing", "dribbling", "defending", "physic"]


data_to_model = data_without_gk.drop(columns=["sofifa_id", "player_url", "short_name", "long_name", "dob", "club_position"])

data_to_predict = gk_data.drop(columns=["sofifa_id", "player_url", "short_name", "long_name", "dob", "club_position", *columns_to_predict])

for column_to_predict in columns_to_predict:

    target = data_to_model[column_to_predict]
    features = data_to_model.drop(columns=columns_to_predict)
    mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    mean_imputer.fit(features)
    imputed_features = mean_imputer.transform(features)

    features_train, features_test, target_train, target_test = train_test_split(imputed_features, target, test_size=0.2)

    linear_regression = LinearRegression()
    linear_regression_model = linear_regression.fit(features_train, target_train)
    linear_regression_score = linear_regression_model.score(features_test, target_test)

    predicted_values = linear_regression_model.predict(mean_imputer.transform(data_to_predict))
    gk_data[column_to_predict] = predicted_values.round()
    data.loc[gk_data.index, column_to_predict] = gk_data[column_to_predict]

    print(f"{column_to_predict}: score: {linear_regression_score:.4f}")

data.loc[gk_data.index]

pace: score: 0.9993
shooting: score: 0.9991
passing: score: 0.9961
dribbling: score: 0.9991
defending: score: 0.9997
physic: score: 0.9991


,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
8,192563,https://sofifa.com/player/192563/bernd-leno/22...,B. Leno,Bernd Leno,83,84,28000000.0,87000.0,1992-03-04,83,190,1.0,GK,1.0,1,3,1,3,0,0,55300000.0,56.0,21.0,38.0,32.0,21.0,55.0,9,9,13,16,9,8,48,35,55,56,52,78,44,57,73,43,70,14,28,25,7,56,23,62,27,15,18,85,80,76,83,85,55.0,35,35,35,32,35,35,35,32,41,41,41,39,43,43,43,39,37,41,41,41,37,36,36,36,36,36,84,30,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,233934,https://sofifa.com/player/233934/aaron-ramsdal...,A. Ramsdale,Aaron Ramsdale,74,79,5000000.0,36000.0,1998-05-14,77,188,1.0,SUB,32.0,1,3,1,1,0,0,9900000.0,49.0,23.0,30.0,30.0,17.0,49.0,15,14,15,18,17,16,33,28,47,50,53,70,49,52,66,35,59,15,35,20,11,47,25,55,18,16,15,75,72,69,71,76,48.0,32,32,32,30,31,31,31,30,34,34,34,33,34,34,34,33,30,33,33,33,30,30,30,30,30,30,75,24,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
10,246671,https://sofifa.com/player/246671/arthur-okonkw...,A. Okonkwo,Arthur Okonkwo,61,77,750000.0,3000.0,2001-09-09,83,196,1.0,RES,33.0,1,2,1,1,0,0,1900000.0,44.0,14.0,20.0,19.0,12.0,39.0,10,6,12,9,12,20,17,17,44,44,40,58,31,41,50,29,50,7,20,11,6,30,19,30,13,13,12,60,58,54,62,63,44.0,24,24,24,21,22,22,22,21,25,25,25,25,24,24,24,25,24,24,24,24,24,24,24,24,24,24,62,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
11,202811,https://sofifa.com/player/202811/emiliano-mart...,E. Martínez,Damián Emiliano Martínez,84,85,33500000.0,81000.0,1992-09-02,88,195,2.0,GK,1.0,1,4,1,3,0,0,66200000.0,58.0,27.0,45.0,31.0,16.0,55.0,22,16,26,13,22,20,51,35,59,57,49,80,49,62,60,40,70,16,37,13,12,66,42,60,11,20,20,85,83,82,83,85,58.0,39,39,39,35,37,37,37,35,43,43,43,41,45,45,45,41,36,40,40,40,36,36,35,35,35,36,85,29,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
12,192317,https://sofifa.com/player/192317/jed-steer/220002,J. Steer,Jed Steer,72,73,1900000.0,31000.0,1992-09-23,80,188,2.0,SUB,12.0,0,2,1,1,0,0,3800000.0,50.0,22.0,28.0,23.0,14.0,48.0,11,11,12,11,12,11,25,22,53,47,37,67,59,53,56,39,61,11,22,22,16,42,41,30,12,12,11

In [24]:
count_na(data)

release_clause_eur     1115
goalkeeping_speed     17054
dtype: int64

## Prevendo goalkeeping_speed dos jogadores

In [25]:
players_data = data.query("GK == 0").copy()
players_data

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
2185,188567,https://sofifa.com/player/188567/pierre-emeric...,P. Aubameyang,Pierre-Emerick Emiliano François Aubameyang,85,85,43500000.0,145000.0,1989-06-18,80,187,1.0,ST,14.0,1,4,4,3,0,0,82700000.0,89.0,85.0,75.0,80.0,35.0,69.0,76,89,73,79,81,76,64,81,87,91,77,86,67,82,75,79,74,78,43,48,90,77,83,87,20,29,36,6,9,15,9,9,NaN,85,85,85,82,83,83,83,82,84,84,84,84,77,77,77,84,67,61,61,61,67,63,54,54,54,63,20,32,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1
2186,209989,https://sofifa.com/player/209989/thomas-partey...,T. Partey,Thomas Partey,83,83,36500000.0,115000.0,1993-06-13,77,185,1.0,RDM,5.0,1,3,3,2,0,0,69400000.0,69.0,71.0,82.0,81.0,80.0,84.0,74,66,68,84,69,70,85,83,63,74,75,82,50,86,83,83,84,78,84,81,70,85,49,86,80,84,80,11,13,9,9,7,NaN,77,77,77,78,78,78,78,78,82,82,82,81,83,83,83,81,82,83,83,83,82,81,83,83,83,81,20,29,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2187,193301,https://sofifa.com/player/193301/alexandre-lac...,A. Lacazette,Alexandre Lacazette,82,82,30000000.0,110000.0,1991-05-28,73,175,1.0,SUB,9.0,1,4,4,3,1,0,57000000.0,75.0,82.0,74.0,82.0,44.0,72.0,68,83,75,83,74,74,70,82,77,73,81,82,82,87,76,66,75,78,73,42,82,75,79,80,39,45,32,11,6,9,5,6,NaN,82,82,82,79,81,81,81,79,82,82,82,81,77,77,77,81,64,65,65,65,64,62,60,60,60,62,18,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2188,222665,https://sofifa.com/player/222665/martin-odegaa...,M. Ødegaard,Martin Ødegaard,82,88,49000000.0,89000.0,1998-12-17,68,178,1.0,CAM,8.0,0,2,5,3,0,0,102900000.0,77.0,74.0,83.0,84.0,58.0,62.0,84,73,52,85,81,83,81,85,80,75,85,74,83,76,46,78,56,77,63,65,76,86,63,76,65,49,51,14,15,15,8,10,NaN,77,77,77,81,79,79,79,81,84,84,84,84,82,82,82,84,75,73,73,73,75,71,63,63,63,71,22,23,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2189,226110,https://sofifa.com/player/226110/nicolas-pepe/...,N. Pépé,Nicolas Pépé,81,82,30500000.0,95000.0,1995-05-29,73,183,1.0,RM,19.0,0,2,4,2,0,0,60200000.0,88.0,79.0,75.0,83.0,36.0,65.0,77,79,57,85,81,79,66,81,88,8

In [26]:
data_to_predict = players_data.drop(columns=["sofifa_id", "player_url", "short_name", "long_name", "dob", "goalkeeping_speed", "club_position"])

In [27]:
features = gk_data.drop(columns=["sofifa_id", "player_url", "short_name", "long_name", "dob", "goalkeeping_speed", "club_position"])
target = gk_data.goalkeeping_speed

mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
mean_imputer.fit(features)
imputed_features = mean_imputer.transform(features)

features_train, features_test, target_train, target_test = train_test_split(imputed_features, target, test_size=0.2)


linear_regression = LinearRegression()
linear_regression_model = linear_regression.fit(features_train, target_train)
linear_regression_score = linear_regression_model.score(features_test, target_test)

print(f"goalkeeping_speed: score: {linear_regression_score:.4f}")

predicted_values = linear_regression_model.predict(mean_imputer.transform(data_to_predict))
players_data["goalkeeping_speed"] = predicted_values.round()
data.loc[players_data.index, "goalkeeping_speed"] = players_data["goalkeeping_speed"]

goalkeeping_speed: score: 0.9993


In [29]:
data.loc[players_data.index]

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
2185,188567,https://sofifa.com/player/188567/pierre-emeric...,P. Aubameyang,Pierre-Emerick Emiliano François Aubameyang,85,85,43500000.0,145000.0,1989-06-18,80,187,1.0,ST,14.0,1,4,4,3,0,0,82700000.0,89.0,85.0,75.0,80.0,35.0,69.0,76,89,73,79,81,76,64,81,87,91,77,86,67,82,75,79,74,78,43,48,90,77,83,87,20,29,36,6,9,15,9,9,91.0,85,85,85,82,83,83,83,82,84,84,84,84,77,77,77,84,67,61,61,61,67,63,54,54,54,63,20,32,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1
2186,209989,https://sofifa.com/player/209989/thomas-partey...,T. Partey,Thomas Partey,83,83,36500000.0,115000.0,1993-06-13,77,185,1.0,RDM,5.0,1,3,3,2,0,0,69400000.0,69.0,71.0,82.0,81.0,80.0,84.0,74,66,68,84,69,70,85,83,63,74,75,82,50,86,83,83,84,78,84,81,70,85,49,86,80,84,80,11,13,9,9,7,69.0,77,77,77,78,78,78,78,78,82,82,82,81,83,83,83,81,82,83,83,83,82,81,83,83,83,81,20,29,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2187,193301,https://sofifa.com/player/193301/alexandre-lac...,A. Lacazette,Alexandre Lacazette,82,82,30000000.0,110000.0,1991-05-28,73,175,1.0,SUB,9.0,1,4,4,3,1,0,57000000.0,75.0,82.0,74.0,82.0,44.0,72.0,68,83,75,83,74,74,70,82,77,73,81,82,82,87,76,66,75,78,73,42,82,75,79,80,39,45,32,11,6,9,5,6,77.0,82,82,82,79,81,81,81,79,82,82,82,81,77,77,77,81,64,65,65,65,64,62,60,60,60,62,18,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2188,222665,https://sofifa.com/player/222665/martin-odegaa...,M. Ødegaard,Martin Ødegaard,82,88,49000000.0,89000.0,1998-12-17,68,178,1.0,CAM,8.0,0,2,5,3,0,0,102900000.0,77.0,74.0,83.0,84.0,58.0,62.0,84,73,52,85,81,83,81,85,80,75,85,74,83,76,46,78,56,77,63,65,76,86,63,76,65,49,51,14,15,15,8,10,80.0,77,77,77,81,79,79,79,81,84,84,84,84,82,82,82,84,75,73,73,73,75,71,63,63,63,71,22,23,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2189,226110,https://sofifa.com/player/226110/nicolas-pepe/...,N. Pépé,Nicolas Pépé,81,82,30500000.0,95000.0,1995-05-29,73,183,1.0,RM,19.0,0,2,4,2,0,0,60200000.0,88.0,79.0,75.0,83.0,36.0,65.0,77,79,57,85,81,79,66,81,

In [28]:
count_na(data)

release_clause_eur    1115
dtype: int64